In [1]:
# Install required packages
!pip install litellm transformers datasets

# Import necessary libraries
import os
import subprocess
import sys
import tempfile
from datasets import load_dataset

print("Dependencies installed and libraries imported successfully.")

  Using cached datasets-2.20.0-py3-none-any.whl.metadata (19 kB)
  Using cached pyarrow-16.1.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (3.0 kB)
  Using cached pyarrow_hotfix-0.6-py3-none-any.whl.metadata (3.6 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tqdm-4.66.4-py3-none-any.whl.metadata (57 kB)
  Using cached xxhash-3.4.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py312-none-any.whl.metadata (7.2 kB)
Using cached datasets-2.20.0-py3-none-any.whl (547 kB)
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
Using cached pyarrow-16.1.0-cp312-cp312-macosx_11_0_arm64.whl (26.0 MB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached tqdm-4.66.4-py3-none-any.whl (78 kB)
Using cached multiprocess-0.70.16-py312-none-any.whl (146 kB)
Using cached pyarrow_hotfix-0.6-py3-none-any.whl (7.9 kB)
Using cached xxhash-3.4.1-cp312-cp312-macosx

In [2]:
# Load the dataset from Hugging Face
dataset = load_dataset("isavita/advent-of-code")

# Filter the dataset for Go solutions
go_solutions = dataset['train'].filter(lambda example: example['solution_lang'] == 'go')

print(f"Loaded {len(go_solutions)} Go solutions from the dataset.")

Generating train split:   0%|          | 0/7893 [00:00<?, ? examples/s]

Filter:   0%|          | 0/7893 [00:00<?, ? examples/s]

Loaded 441 Go solutions from the dataset.


In [13]:
class GoConstants:
    @staticmethod
    def file_ext():
        return "go"
    
    @staticmethod
    def lang():
        return "go"
    
    @staticmethod
    def compiler():
        return "go"
    
    @staticmethod
    def runtime():
        return "/opt/homebrew/bin/go run"

print("Go constants class defined.")

Go constants class defined.


In [6]:
import tempfile
import subprocess
import os

def test_go_execution():
    # Hello World program in Go
    go_code = '''
package main

import "fmt"

func main() {
    fmt.Println("Hello, World!")
}
'''
    
    # Create a temporary file
    with tempfile.NamedTemporaryFile(suffix=f'.{GoConstants.file_ext()}', delete=False, mode='w') as temp_file:
        temp_file.write(go_code)
        temp_file_path = temp_file.name
    
    try:
        # Run the Go program
        result = subprocess.run(f"{GoConstants.runtime()} {temp_file_path}", shell=True, capture_output=True, text=True)
        
        if result.returncode == 0:
            print("Go program executed successfully.")
            print("Output:", result.stdout.strip())
        else:
            print("Error executing Go program:")
            print(result.stderr)
    finally:
        # Clean up the temporary file
        os.unlink(temp_file_path)

# Run the test
test_go_execution()

Go program executed successfully.
Output: Hello, World!


In [8]:
import litellm
import warnings

# Suppress the specific warning
warnings.filterwarnings("ignore", message="Field .* has conflict with protected namespace .*")

class LLMWrapper:
    def __init__(self, model_name, api_base=None):
        self.model_name = model_name
        self.api_base = api_base
        self.temperature = 0.3  # Hardcoded temperature as specified

    def generate(self, system_prompt, user_prompt):
        try:
            completion_params = {
                "model": self.model_name,
                "messages": [
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                "temperature": self.temperature,
            }
            
            if self.api_base:
                completion_params["api_base"] = self.api_base

            response = litellm.completion(**completion_params)
            return response.choices[0].message.content
        except Exception as e:
            print(f"Error in LLM API call: {str(e)}")
            return None

# Test the LLMWrapper with the local Ollama model
def test_llm_wrapper():
    wrapper = LLMWrapper("ollama/llama3:8b-instruct-q5_K_M", api_base="http://localhost:11434")
    
    system_prompt = "You are a helpful assistant."
    user_prompt = "Tell me a short joke about programming."
    
    print("Testing with local Ollama model:")
    response = wrapper.generate(system_prompt, user_prompt)
    if response:
        print("Local LLM Response:")
        print(response)
    else:
        print("Failed to get a response from the local LLM.")

# Run the test
test_llm_wrapper()

Testing with local Ollama model:
Local LLM Response:
Here's one:

Why do programmers prefer dark mode?

Because light attracts bugs!

Hope that made you giggle!


In [11]:
import threading
import time
import queue

class TimeoutRunner:
    @staticmethod
    def run_with_timeout(func, args=(), kwargs={}, timeout_duration=10):
        result_queue = queue.Queue()

        def wrapper():
            try:
                result = func(*args, **kwargs)
                result_queue.put(result)
            except Exception as e:
                result_queue.put(e)

        thread = threading.Thread(target=wrapper)
        thread.daemon = True
        thread.start()
        
        try:
            return result_queue.get(timeout=timeout_duration)
        except queue.Empty:
            return None

# Test the TimeoutRunner
def test_timeout_runner():
    def quick_function():
        return "Quick function completed"

    def slow_function():
        time.sleep(5)
        return "Slow function completed"

    runner = TimeoutRunner()

    print("Testing quick function (should complete):")
    result = runner.run_with_timeout(quick_function, timeout_duration=2)
    print(result)

    print("\nTesting slow function (should timeout):")
    result = runner.run_with_timeout(slow_function, timeout_duration=2)
    print(result)

# Run the test
test_timeout_runner()

Testing quick function (should complete):
Quick function completed

Testing slow function (should timeout):
None


In [22]:
import os
import tempfile
import subprocess

class EvaluateCode:
    def __init__(self, code, input_data):
        self.code = code
        self.input_data = input_data

    def run(self):
        with tempfile.TemporaryDirectory() as temp_dir:
            # Save code to a file using the extension from GoConstants
            code_file = os.path.join(temp_dir, f"main.{GoConstants.file_ext()}")
            with open(code_file, 'w') as f:
                f.write(self.code)

            # Save input to a file
            input_file = os.path.join(temp_dir, "input.txt")
            with open(input_file, 'w') as f:
                f.write(self.input_data)

            # Directly run the Go file
            run_command = f"{GoConstants.runtime()} {code_file} < {input_file}"

            # Run the command in the temporary directory
            run_result = subprocess.run(
                run_command,
                shell=True,
                capture_output=True,
                text=True,
                cwd=temp_dir  # Set the working directory to the temporary directory
            )

            if run_result.returncode != 0:
                return f"Runtime error: {run_result.stderr}"

            return run_result.stdout

# Test the EvaluateCode class
def test_evaluate_code():
    code = '''
package main

import (
    "fmt"
    "bufio"
    "os"
    "strconv"
)

func main() {
    scanner := bufio.NewScanner(os.Stdin)
    scanner.Scan()
    n, _ := strconv.Atoi(scanner.Text())
    fmt.Printf("The number is: %d", n)
}
'''
    input_data = "43"

    evaluator = EvaluateCode(code, input_data)
    result = evaluator.run()
    print("Evaluation result:")
    print(result)

# Run the test
test_evaluate_code()

Evaluation result:
The number is: 43


In [23]:
import re
import os
import json
from pathlib import Path

class ChallengeSolver:
    def __init__(self, model_name, llm_wrapper):
        self.model_name = model_name
        self.llm_wrapper = llm_wrapper

    def extract_code(self, input_text):
        pattern = rf"```(?:{GoConstants.lang()})?(.*?)```"
        matches = re.findall(pattern, input_text, re.DOTALL)
        return matches[0] if matches else input_text

    def valid_solution(self, result, answer):
        special_cases = [
            ".##..####.###..#..#.###..####.###....##.###...###.",
            " ##  #### ###  #  # ###  #### ###    ## ###   ### ",
            "#....#..#....#.....###..######....##....#....#....##....######",
            "#    #  #    #     ###  ######    ##    #    #    ##    ######",
            "3.465154e+06", "3.465154e+6",
            "####.###..####.#..#.###..\n#....#..#....#.#..#.#..#.",
            "#### ###  #### #  # ###  \n#    #  #    # #  # #  # ",
            ".#....###....##.#..#.####.#..#.#....#..#.\n",
            " #    ###    ## #  # #### #  # #    #  # \n",
            " █    ███    ██ █  █ ████ █  █ █    █  █ \n",
            "#..#.#..#.#..#.#..#.#..#.#..#.#..#....#",
            "#  # #  # #  # #  # #  # #  # #  #    #",
            "###..###..###...##..###...##...##..####.",
            "###  ###  ###   ##  ###   ##   ##  #### "
        ]
        return answer in result or any(case in result for case in special_cases)

    def solve(self, challenge_data):
        system_prompt = f"You are an expert Go programmer solving Advent of Code challenges. Write a Go solution for the following problem:"
        user_prompt = f"Day {challenge_data['name']}: {challenge_data['task']}\n\nProvide only the Go code solution, no explanations."

        max_attempts = 3
        for attempt in range(max_attempts):
            response = self.llm_wrapper.generate(system_prompt, user_prompt)
            code = self.extract_code(response)

            evaluator = EvaluateCode(code, challenge_data['input'])
            result = evaluator.run()

            if self.valid_solution(result, challenge_data['answer']):
                self.save_solution(challenge_data, code)
                return True, result

        return False, "Failed to find a valid solution after multiple attempts."

    def save_solution(self, challenge_data, code):
        home_dir = str(Path.home())
        file_path = f"{home_dir}/code/advent_generated/{challenge_data['name']}.{GoConstants.file_ext()}"
        os.makedirs(os.path.dirname(file_path), exist_ok=True)
        with open(file_path, 'w') as f:
            f.write(code)

# Test the ChallengeSolver class
def test_challenge_solver():
    class MockLLMWrapper:
        def generate(self, system_prompt, user_prompt):
            return '''Here's a Go solution for the problem:

```go
package main

import (
    "fmt"
    "os"
    "strings"
)

func main() {
    data, err := os.ReadFile("input.txt")
    if err != nil {
        fmt.Println("File reading error", err)
        return
    }

    input := strings.TrimSpace(string(data))
    sum := 0

    for i := 0; i < len(input); i++ {
        next := (i + 1) % len(input)
        if input[i] == input[next] {
            sum += int(input[i] - '0')
        }
    }

    fmt.Println(sum)
}
```'''

    # Load the challenge data from the JSON
    challenge_data = json.loads('''{
        "name": "day1_part1_2017",
        "task": "--- Day 1: Inverse Captcha ---\\nThe night before Christmas, one of Santa's Elves calls you in a panic. \\"The printer's broken! We can't print the Naughty or Nice List!\\" By the time you make it to sub-basement 17, there are only a few minutes until midnight. \\"We have a big problem,\\" she says; \\"there must be almost fifty bugs in this system, but nothing else can print The List. Stand in this square, quick! There's no time to explain; if you can convince them to pay you in stars, you'll be able to--\\" She pulls a lever and the world goes blurry.\\n\\nWhen your eyes can focus again, everything seems a lot more pixelated than before. She must have sent you inside the computer! You check the system clock: 25 milliseconds until midnight. With that much time, you should be able to collect all fifty stars by December 25th.\\n\\nCollect stars by solving puzzles. Two puzzles will be made available on each day millisecond in the Advent calendar; the second puzzle is unlocked when you complete the first. Each puzzle grants one star. Good luck!\\n\\nYou're standing in a room with \\"digitization quarantine\\" written in LEDs along one wall. The only door is locked, but it includes a small interface. \\"Restricted Area - Strictly No Digitized Users Allowed.\\"\\n\\nIt goes on to explain that you may only leave by solving a captcha to prove you're not a human. Apparently, you only get one millisecond to solve the captcha: too fast for a normal human, but it feels like hours to you.\\n\\nThe captcha requires you to review a sequence of digits (your puzzle input) and find the sum of all digits that match the next digit in the list. The list is circular, so the digit after the last digit is the first digit in the list.\\n\\nFor example:\\n\\n1122 produces a sum of 3 (1 + 2) because the first digit (1) matches the second digit and the third digit (2) matches the fourth digit.\\n1111 produces 4 because each digit (all 1) matches the next.\\n1234 produces 0 because no digit matches the next.\\n91212129 produces 9 because the only digit that matches the next one is the last digit, 9.\\nWhat is the solution to your captcha?",
        "input": "111831362354551173134957758417849716877188716338227121869992652972154651632296676464285261171625892888598738721925357479249486886375279741651224686642647267979445939836673253446489428761486828844713816198414852769942459766921928735591892723619845983117283575762694758223956262583556675379533479458964152461973321432768858165818549484229241869657725166769662249574889435227698271439423511175653875622976121749344756734658248245212273242115488961818719828258936653236351924292251821352389471971641957941593141159982696396228218461855752555358856127582128823657548151545741663495182446281491763249374581774426225822474112338745629194213976328762985884127324443984163571711941113986826168921187567861288268744663142867866165546795621466134333541274633769865956692539151971953651886381195877638919355216642731848659649263217258599456646635412623461138792945854536154976732167439355548965778313264824237176152196614333748919711422188148687299757751955297978137561935963366682742334867854892581388263132968999722366495346854828316842352829827989419393594846893842746149235681921951476132585199265366836257322121681471877187847219712325933714149151568922456111149524629995933156924418468567649494728828858254296824372929211977446729691143995333874752448315632185286348657293395339475256796591968717487615896959976413637422536563273537972841783386358764761364989261322293887361558128521915542454126546182855197637753115352541578972298715522386683914777967729562229395936593272269661295295223113186683594678533511783187422193626234573849881185849626389774394351115527451886962844431947188429195191724662982411619815811652741733744864411666766133951954595344837179635668177845937578575117168875754181523584442699384167111317875138179567939174589917894597492816476662186746837552978671142265114426813792549412632291424594239391853358914643327549192165466628737614581458189732579814919468795493415762517372227862614224911844744711698557324454211123571327224554259626961741919243229688684838813912553397698937237114287944446722919198743189848428399356842626198635297851274879128322358195585284984366515428245928111112613638341345371",
        "answer": "1044",
        "solution": "package main\\n\\nimport (\\n\\t\\"fmt\\"\\n\\t\\"os\\"\\n\\t\\"strings\\"\\n)\\n\\nfunc main() {\\n\\tdata, err := os.ReadFile(\\"input.txt\\")\\n\\tif err != nil {\\n\\t\\tfmt.Println(\\"File reading error\\", err)\\n\\t\\treturn\\n\\t}\\n\\n\\tinput := strings.TrimSpace(string(data))\\n\\tsum := 0\\n\\n\\tfor i := 0; i < len(input); i++ {\\n\\t\\tnext := (i + 1) % len(input)\\n\\t\\tif input[i] == input[next] {\\n\\t\\t\\tsum += int(input[i] - '0')\\n\\t\\t}\\n\\t}\\n\\n\\tfmt.Println(sum)\\n}",
        "solution_lang": "go",
        "year": 2017
    }''')

    solver = ChallengeSolver("gpt-3.5-turbo", MockLLMWrapper())
    success, result = solver.solve(challenge_data)

    print(f"Solution found: {success}")
    print(f"Result: {result}")

# Run the test
test_challenge_solver()

Solution found: True
Result: 1044



In [24]:
import time
from datasets import load_dataset

def run_max_times(func, max_attempts=1):
    for _ in range(max_attempts):
        success, result = func()
        if success:
            return True, result
    return False, "Failed after maximum attempts"

def main():
    # Load the dataset
    dataset = load_dataset("isavita/advent-of-code")
    go_solutions = dataset['train'].filter(lambda example: example['solution_lang'] == 'go')

    # Initialize the LLM wrapper and solver
    llm_wrapper = LLMWrapper("gpt-4-turbo-2024-04-09")  # You can change this to your preferred model
    solver = ChallengeSolver("gpt-4-turbo-2024-04-09", llm_wrapper)

    total_challenges = len(go_solutions[:1])
    print(f"Total challenges: {total_challenges}")

    for i, challenge in enumerate(go_solutions, 1):
        def solve_attempt():
            return solver.solve(challenge)

        success, result = run_max_times(solve_attempt, max_attempts=1)  # You can adjust max_attempts as needed

        if success:
            print(f"Solved {challenge['name']} successfully!")
        else:
            print(f"Failed to solve {challenge['name']}")

        progress = (i / total_challenges) * 100
        print(f"Progress: {progress:.1f}%")
        
        time.sleep(15)  # Sleep for 15 seconds between attempts

if __name__ == "__main__":
    main()

Filter:   0%|          | 0/7893 [00:00<?, ? examples/s]

Total challenges: 7
Solved day1_part1_2015 successfully!
Progress: 14.3%
Solved day1_part2_2015 successfully!
Progress: 28.6%
Solved day2_part1_2015 successfully!
Progress: 42.9%
Solved day2_part2_2015 successfully!
Progress: 57.1%


KeyboardInterrupt: 